In [1]:
from datetime import *
from dateutil.relativedelta import relativedelta
import pandas as pd
import tushare as ts
from math import *
import statsmodels.api as sm
from collections import OrderedDict #保持Key的顺序
import matplotlib.pyplot as plt
import seaborn as sns
from time import sleep
sns.set_style('whitegrid')

In [2]:
token='ec42e45eba5526cd886eec58e37f084fc9373e353ba829132a23a456'
pro = ts.pro_api(token)#初始化pro接口

In [3]:
def remove_st_stock():
    df = pro.query('stock_basic', exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
    df=df[~df.name.str.contains('ST')]
    df['list_date'] = pd.to_datetime(df['list_date']).dt.date#,format='%Y%m%d'
    return df

In [4]:
def get_day_offset_stock(context, start_day, day_offset):
    offset = start_day - relativedelta(months=day_offset)
    context = context[context['list_date'] <= offset]
    return context

In [12]:
#获取当前时点的股票
def get_stocks(trDate):
    A_stocks=remove_st_stock() #提取非ST，非PT的全部A股
    status=get_day_offset_stock(A_stocks, trDate, 1) #提取股票的交易状态和首发上市日期
    trade_codes=status['ts_code'] #选取交易且上市时间超过30天的股票,并将index转换成list格式
    return trade_codes

In [6]:
##测试区间：2005年1月1日至2017年12月31日，取每月月初的开盘价和每月月末的收盘价 
def get_beg_end_month(beg_date,end_date):
    month_beg_add=w.tdaysoffset(-1,beg_date,"Period=M").Data[0][0].strftime("%Y-%m-%d") #将首月往前推一个月,再取该日对应的月末日期，再往后推一日记得到首月的月初日期
    month_ends=w.tdays(month_beg_add,end_date, "Period=M").Data[0] 
    month_ends= [dt.strftime("%Y-%m-%d") for dt in month_ends]  #获取每月最后一个交易日
    dates=month_ends[1:]   #提取股票时间,即测试区间内每月月末交易日对应的股票,该日期也为测试区间的每月月末交易日
    month_begs=[w.tdaysoffset(1,i,"").Data[0][0].strftime("%Y-%m-%d") for i in month_ends][:-1] #将月末交易日往后推一日，得到下一个月月初交易日
    return month_begs,dates

In [16]:
#获取月初开盘价和月末收盘价
def get_price(stock_list, month_begs,month_ends):
    dict_df = OrderedDict()
    df = pd.DataFrame()
    count = 0
    for stock in stock_list:
        count +=1
        if(count == 120):
            sleep(60)
            count = 0
        else:
            info = pro.monthly(ts_code=stock, start_date=month_begs, end_date=month_ends, fields='trade_date, ts_code,open,close')
            df=df.append(info)#获取每月月初月末的收盘价
    print(df)
    #dict_df[date] = df  #将得到的每一期的开盘价和收盘价，以时间为键，以价格为值，有序存入dict_df字典中
    price= df.groupby(df['trade_date'])   #将字典对应的多个列表合并为一个列表
    
    return price

In [17]:
stock_list = get_stocks(datetime.today().date())
get_price(stock_list, '20100101','20200101')

      ts_code trade_date  close   open
0   000001.SZ   20191231  16.45  15.35
1   000001.SZ   20191129  15.29  16.35
2   000001.SZ   20191031  16.26  15.60
3   000001.SZ   20190930  15.59  14.15
4   000001.SZ   20190830  14.16  14.06
..        ...        ...    ...    ...
4   688388.SH   20190830  57.62  65.10
5   688388.SH   20190731  65.86  66.00
0   688389.SH   20191231  16.42  16.53
1   688389.SH   20191129  16.34  22.00
0   688399.SH   20191231  57.90  52.33

[291070 rows x 4 columns]


In [ ]:
stock_list.head()

In [9]:
pro.monthly(ts_code=stock_list[0], start_date='20100101', end_date='20200101', fields='trade_date, ts_code,open,close')

,ts_code,trade_date,close,open
